# Reduccion de variables


In [1]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [2]:
# %load ../../standard_import.txt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
from test_helper import Test

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)
 
#%config InlineBackend.figure_formats = {'pdf',}
%matplotlib inline

from sklearn import metrics, neighbors, tree, preprocessing
from scipy import stats

from matplotlib.colors import ListedColormap

from shutil import copyfile

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn import neighbors, model_selection, tree
from sklearn.preprocessing import PolynomialFeatures

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_seq_items', None)

# Se importa la librería de selección de variables
from sklearn import feature_selection
from sklearn.feature_selection import SelectPercentile

### Datos

In [3]:
dataset = pd.read_csv('all_stocks_5yr.csv', delimiter=',')
#se eliminan las filas que tienen valores nulos de cara a realizar posteriores estudios
#HABRIA QUE TENERLO IMPUTADO CON VALORES PERDIDOS!!
dataset.dropna(axis=0, inplace = True)

atSalida = 'close'
atEntrada = ['open','high','low','volume']

dataset_output = dataset[atSalida].copy()
dataset.drop(atSalida, axis=1, inplace=True)
dataset = dataset[atEntrada].copy()

# SUSTITUIR POR LO BUENO
X_train, X_test, y_train, y_test = train_test_split(dataset, dataset_output, test_size=0.2, random_state=12)

In [ ]:
pipePolinomial = Pipeline([('polynomial', PolynomialFeatures(degree=2)),('scaler', StandardScaler()),('regressor', LinearRegression())])

### Seleccion de variables

* Correlación mutua: función [*f_regression*](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.f_regression.html#sklearn.feature_selection.f_regression)
* Información mutua: función [*mutual_info_regression*](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.mutual_info_regression.html#sklearn.feature_selection.mutual_info_regression)


Una vez conocida la calidad de cada variable se deben escoger las mejores. Para ello vimos en las clases teóricas que había varias opciones. La librería Scikit-learn ofrece dos de estas técnicas en forma de clases (con sus campos y sus métodos):
* Elegir las k mejores con la clase [*SelectKBest*](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectKBest.html#sklearn.feature_selection.SelectKBest)
* Elegir las variables en base al percentil (el % de las variables) con la clase [*SelectPercentile*](http://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectPercentile.html#sklearn.feature_selection.SelectPercentile)



 Correlación mutua: función [*f_regression*]

In [4]:
# Se crea la Pipeline con las fases deseadas de Correlación mutua
pipe_f_regression = Pipeline([
    ('correlacion_mutua', feature_selection.SelectPercentile(feature_selection.f_regression)),
    ('polynomial', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())])
# Se crea el grid de híper-parámetros
parameters_f_regression = {'correlacion_mutua__percentile': [10, 20, 30]}

* Información mutua: función [*mutual_info_regression*]

In [7]:
# Se crea la Pipeline con las fases deseadas de Informacion mutua
pipe_f_regression = Pipeline([
    ('informacion_mutua', feature_selection.SelectPercentile(feature_selection.mutual_info_regression)),
    ('polynomial', PolynomialFeatures(degree=2)),
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())])
# Se crea el grid de híper-parámetros
parameters_f_regression = {'informacion_mutua__percentile': [10, 20, 30]}

In [ ]:
# Selecciona la pipe a utilizar
pipe = pipe_f_regression
parameters = parameters_f_regression

# Se llama al constructor de GridSearchCV para que genere todas las combinaciones de los híper-parámetros definidos anteriormente
gridSearch_pipe = model_selection.GridSearchCV(pipe,parameters,cv=3,return_train_score=True)
# Se realiza el aprendizaje de todos los clasificadores considerados (todas las configuraciones)
gridSearch_pipe = gridSearch_pipe.fit(X_train,y_train.values.ravel())
# Se muestra la mejor configuración junto con su rendimiento
print(gridSearch_pipe.best_score_)
print(gridSearch_pipe.best_params_)


# Almacenamos el DataFrame con los resultados
diccionarioResultados = gridSearch_pipe.cv_results_

# Se imprime el mejor porcentaje de acierto y los resultados de todas las configuraciones
print(gridSearch_pipe.best_score_,gridSearch_pipe.best_params_)

# Se obtiene el rendimiento en entrenamiento y test por la mejor configuración
accTrain = gridSearch_pipe.score(X_train,y_train.values.ravel())*100
accTest = gridSearch_pipe.score(X_test,y_test.values.ravel())*100
print('Resultado en entrenamiento: {}%'.format(accTrain))
print('Resultado en test: {}%'.format(accTest))

PCA

In [ ]:
# Se crea la Pipeline con las fases deseadas para el pipe PCA
pipe_pca = Pipeline([('pca', PCA(svd_solver='full')),
                 ('polynomial', PolynomialFeatures(degree=2)),
                 ('scaler', StandardScaler()),
                 ('regressor', LinearRegression())])
# Se crea el grid de híper-parámetros
parameters_pca = {'pca__n_components': [0.10, 0.30, 0.50, 0.70, 0.90]}
# Se llama al constructor de GridSearchCV para que genere todas las combinaciones de los híper-parámetros definidos anteriormente
gridSearch_pipe = model_selection.GridSearchCV(pipe_pca,parameters_pca,cv=10,n_jobs=-1,return_train_score=True)
# Se realiza el aprendizaje de todos los clasificadores considerados (todas las configuraciones)
gridSearch_pipe =  gridSearch_pipe.fit(X_train,y_train)
# Se muestra la mejor configuración junto con su rendimiento
print(gridSearch_pipe.best_score_)
print(gridSearch_pipe.best_params_)


# Almacenamos el DataFrame con los resultados
diccionarioResultados = gridSearch_pipe.cv_results_

# Se imprime el mejor porcentaje de acierto y los resultados de todas las configuraciones
print(gridSearch_pipe.best_score_,gridSearch_pipe.best_params_)

# Se obtiene el rendimiento en entrenamiento y test por la mejor configuración
accTrain = gridSearch_pipe.score(X_train,y_train.values.ravel())*100
accTest = gridSearch_pipe.score(X_test,y_test.values.ravel())*100
print('Resultado en entrenamiento: {}%'.format(accTrain))
print('Resultado en test: {}%'.format(accTest))

### Seleccióndeinstancias

Utilizamos Wrapers ya que miran la precision con Leave One Out y no tiene en cuanta las clases, las cuales no nos importan ennuestro modelo de regresion

In [ ]:
def RMHC(regresor, X, y, s=0.1, iteraciones=1000):
    """
     Algoritmo RMHC (Random Mutation Hill Climbing) para la selección de instancias.
      Se comienza con una selección aleatoria de s * nEjemplos instancias. Para cada iteración, se elige una instancia
      seleccionada y una no seleccionada para ser intercambiadas. Si el intercambio mejora la precisión (leave-one-out) sobre train
      se mantiene el cambio, sino se deshace
    :param X: Matriz con los ejemplos de entrenamiento (se asume que los ejemplos están normalizados)
    :param y: Vector con la salida de los ejemplos en X
    :param s: Porcentaje de instancias a ser seleccionadas (s es un valor entre 0 y 1)
    :param iteraciones: Número de iteraciones (intercambios) a probar
    :return: Vector con la máscara de instancias seleccionadas 
            (La posición S[i]=True indica que la instancia i ha sido seleccionada y False lo contrario)
    """
    # Como usamos aleatorios en RMHC establecemos la semilla para que el test sea correcto y siempre obtengamos el mismo resultado
    np.random.seed(12312)

    # En este caso vamos a mantener dos vectores de enteros con los índices de las instancias seleccionadas y no seleccionadas
    # Establecemos el número de instancias a seleccionar (es fijo)
    nSel = int(X.shape[0]*s)
    
    # Obtenemos una permutación aleatoria de todos los índices disponibles 
    # (un array de 0 a X.shape[0] con valores ordenados aleatoriamente): utilizad permutation de Numpy
    permute = np.random.permutation(X.shape[0])
    
    # Cogemos como seleccionadas las instancias correspondientes a los primeros nSel índices  y no seleccionadas el resto#     
    seleccionadas = permute[:nSel]
    noSeleccionadas = permute[nSel:]

    # Calculamos la preicisión con la selección inicial
    # Entrenamos regresor con las instancias seleccionadas
    regresor.fit(X[seleccionadas], y[seleccionadas])
    
    # Obtenemos las salidas con leaveOneOut (para no tener en cuenta las instancias seleccionadas como vecinos de sí mismas)
    salidas = regresor.predict(X)
        
    # Calculamos la precisión
    acc = metrics.accuracy_score(salidas, y)

    # Comenzamos las iteraciones:
    #   En cada una, intercambiamos una instancia seleccionada con una que no lo está y comprobamos si mejora la precisión
    for i in range(0, iteraciones):
        # Seleccionamos un índice de manera aleatoria que nos indica la instancia a eliminar de las seleccionadas: randint de Numpy
        quitar = np.random.randint(seleccionadas.size)
        
        # Seleccionamos un índice de manera aleatoria que nos indica la instancia a añadir a las seleccionadas: randint de Numpy
        poner = np.random.randint(noSeleccionadas.size)
        
        # Guardamos el ejemplo eliminado
        aux =  seleccionadas[quitar]
        
        # Añadimos la nueva instancia seleccionada sustituyendo la que se elimina: intercambio en las listas de seleccionados y no seleccionados
        seleccionadas[quitar] = noSeleccionadas[poner]
        
        # Entrenamos de nuevo regresor
        regresor.fit(X[seleccionadas], y[seleccionadas])

        # Calculamos la precisión de la solución actual
        # Obtenemos las salidas con leaveOneOut (para no tener en cuenta las instancias seleccionadas como vecinos de sí mismas)
        salidas = regresor.predict(X)
        
        # Calculamos la precisión
        accNew = metrics.accuracy_score(salidas, y)

        # Si la precisión actual es peor que la anterior, devolvemos la instancia eliminada a las seleccionadas
        # Sino si la precisión de la solución actual es mejor o igual que la anterior, guardamos la precisión y
        #  añadimos la instancia eliminada a las no seleccionadas
        if accNew >= acc:
            acc = accNew
            noSeleccionadas[poner] = aux
        else:
            seleccionadas[quitar] = aux

        if i % 100 == 0:
            print("precision en iteracion {}: {}".format(i, acc))

    # A partir de las seleccionadas creamos la máscara de instancias seleccionadas 
    # donde True en la posición i indica que la instancia i es seleccionada
    S = np.zeros(len(y), bool)
    S[seleccionadas] = True

    return S

### Generacióndevariableseinstancias 